In [1]:
import random
import pyspark.sql.functions as F

In [2]:
nRegisters = 100000
header = ['cod_estado', 'sexo', 'estado_civil']
var = [[x,y,z] for x,y,z in  zip([random.choice(range(11,54)) for _ in range(nRegisters)], [random.choice(['F', 'M']) for _ in range(nRegisters)], [random.choice(['SOLTEIRO', 'CASADO', 'DIVORCIADO']) for _ in range(nRegisters)])]
groupby_var = 'cod_estado'
base_name = 'base1'
df = spark.createDataFrame(var, header)

In [3]:
df.show()

+----------+----+------------+
|cod_estado|sexo|estado_civil|
+----------+----+------------+
|        18|   F|      CASADO|
|        52|   F|      CASADO|
|        17|   M|    SOLTEIRO|
|        42|   F|  DIVORCIADO|
|        25|   M|      CASADO|
|        26|   F|  DIVORCIADO|
|        50|   M|  DIVORCIADO|
|        31|   F|      CASADO|
|        41|   M|    SOLTEIRO|
|        29|   M|  DIVORCIADO|
|        22|   F|    SOLTEIRO|
|        15|   M|  DIVORCIADO|
|        46|   F|  DIVORCIADO|
|        24|   F|    SOLTEIRO|
|        39|   M|      CASADO|
|        38|   M|    SOLTEIRO|
|        19|   F|  DIVORCIADO|
|        21|   M|  DIVORCIADO|
|        37|   F|    SOLTEIRO|
|        30|   M|      CASADO|
+----------+----+------------+
only showing top 20 rows



In [4]:
agg = df.groupby(header).count()
agg_concat = agg.withColumn('agg_concat', F.concat_ws('_', *[x for x in header if x != groupby_var]))
agg_concat = agg_concat.withColumn('agg_concat', F.concat_ws('_', F.lit(base_name), 'agg_concat'))

In [5]:
agg_concat.show()

+----------+----+------------+-----+------------------+
|cod_estado|sexo|estado_civil|count|        agg_concat|
+----------+----+------------+-----+------------------+
|        26|   F|      CASADO|  369|    base1_F_CASADO|
|        40|   F|  DIVORCIADO|  391|base1_F_DIVORCIADO|
|        16|   F|  DIVORCIADO|  378|base1_F_DIVORCIADO|
|        22|   F|      CASADO|  414|    base1_F_CASADO|
|        12|   F|      CASADO|  398|    base1_F_CASADO|
|        37|   F|    SOLTEIRO|  385|  base1_F_SOLTEIRO|
|        51|   F|      CASADO|  385|    base1_F_CASADO|
|        48|   M|      CASADO|  383|    base1_M_CASADO|
|        18|   F|    SOLTEIRO|  416|  base1_F_SOLTEIRO|
|        14|   M|  DIVORCIADO|  451|base1_M_DIVORCIADO|
|        41|   M|  DIVORCIADO|  356|base1_M_DIVORCIADO|
|        25|   M|  DIVORCIADO|  417|base1_M_DIVORCIADO|
|        37|   F|      CASADO|  439|    base1_F_CASADO|
|        46|   F|  DIVORCIADO|  371|base1_F_DIVORCIADO|
|        22|   M|  DIVORCIADO|  398|base1_M_DIVO

In [6]:
result = agg_concat.groupby(groupby_var).pivot('agg_concat').sum('count').orderBy(groupby_var)

In [7]:
result.toPandas()

,cod_estado,base1_F_CASADO,base1_F_DIVORCIADO,base1_F_SOLTEIRO,base1_M_CASADO,base1_M_DIVORCIADO,base1_M_SOLTEIRO
0,11,412,389,370,417,373,347
1,12,398,352,450,370,391,385
2,13,358,396,387,399,383,426
3,14,403,382,408,376,451,370
4,15,370,389,366,425,404,395
5,16,387,378,402,392,381,424
6,17,421,393,360,352,384,403
7,18,350,416,416,372,390,386
8,19,404,414,370,371,381,367
9,20,396,387,439,383,403,412
